# This NB is for showing how to submit a compilation for GPUs and CPUs and subsequent submit for running the programs.
# Works basically also from the terminal
# Please use the jobscripts from this NB for your own programs
- one bash file for compilation 
- one bash file for running the programs

Below is just a sample sycl program to test.It prints out the selected device.

In [29]:
%%file simple-sycl-app.cpp

#include <sycl/sycl.hpp>
#include <iostream>

using namespace sycl;

int main() {

  std::cout << "running program:  simple-sycl-app \n" ;

  queue Q;    

  std::cout << "******************************************* \n";  
  std::cout << "Selected device: " <<
    Q.get_device().get_info<info::device::name>() << "\n";
  std::cout << "******************************************* \n";

  return 0;
}


Overwriting simple-sycl-app.cpp


# compiling for nvidia gpu and our amd cpus

In [39]:
%%file job_vsc5_compile.sh
#!/bin/bash

#SBATCH --job-name    your_jobname
#SBATCH --cluster     vsc5
#SBATCH --reservation training_gpu
#SBATCH --qos         zen3_0512_a100x2_training
#SBATCH --partition   zen3_0512_a100x2
#SBATCH --gres        gpu:1
#SBATCH --time        00:01:00

###########################

spack unload
spack load cuda@11.8.0

source /opt/sw/vsc4/VSC/x86_64/generic/intel/oneapi/setvars.sh 
export LD_LIBRARY_PATH=/gpfs/opt/sw/cuda-zen/spack-0.19.0/opt/spack/linux-almalinux8-zen/gcc-9.5.0/gcc-12.2.0-nu5le4qn6edhcjiocq7wddclrvdj4xfg/lib64:$LD_LIBRARY_PATH    

###########################

echo "**********************************************"
echo "output of nvidia-smi"
echo "**********************************************"
nvidia-smi


echo "**********************************************"
echo "output of sycl-ls"
echo "**********************************************"
sycl-ls


echo "**********************************************"
echo "compiling "
echo "icpx -fsycl -fsycl-targets=nvptx64-nvidia-cuda,spir64_x86_64 simple-sycl-app.cpp -o simple-sycl-app"
echo "**********************************************"
icpx -fsycl -fsycl-targets=nvptx64-nvidia-cuda simple-sycl-app.cpp -o simple-sycl-app 


Overwriting job_vsc5_compile.sh


In [46]:
! sbatch job_vsc5_compile.sh

sbatch: Allocating 50.0 % of cpu resources: 64 / 128.
sbatch: Number of tasks adjusted to 64.
Submitted batch job 1620837 on cluster vsc5


#### the $USER is for your username

In [48]:
! squeue -u $USER --cluster vsc5

CLUSTER: vsc5
             JOBID            PARTITION     NAME     USER ST       TIME  NODES     NODELIST(REASON)
           1620734    zen3_0512_jupyter vsc5_jh_ trainee9  R    2:36:34      1            n3503-022


In [43]:
# insert the jobid 
!cat slurm-<JOBID>.out

/bin/bash: JOBID: No such file or directory


In [49]:
!ls -ltr 

total 6851
-rw-r--r-- 1 trainee96 p70824  420336 Dec  5 11:46 1_1_welcome_FINAL.pdf
-rw-r--r-- 1 trainee96 p70824 1833724 Dec  7 13:27 1_2_Intel_Devcloud_FINAL.pdf
-rw-r--r-- 1 trainee96 p70824 3678052 Dec  8 11:46 3_adv_sycl_sonersteiner_vienna_FINAL.pdf
-rw-r--r-- 1 trainee96 p70824  268829 Dec  8 13:33 SYCL_Introduction.ipynb
-rw-r--r-- 1 trainee96 p70824  738212 Dec  8 13:33 SYCL_Introduction.pdf
-rw-r--r-- 1 trainee96 p70824     762 Dec 11 06:00 job_vsc5_gpu_run.sh
-rw-r--r-- 1 trainee96 p70824     401 Dec 11 06:14 simple-sycl-app.cpp
drwxrwxr-x 7 trainee96 p70824    4096 Dec 11 06:22 1445-1600-LAB2-advanced-sycl
-rw-r--r-- 1 trainee96 p70824   22316 Dec 11 07:28 1_3_job_submission_vsc5.ipynb
-rw-r--r-- 1 trainee96 p70824    1201 Dec 11 08:27 job_vsc5_compile.sh
-rw-r--r-- 1 trainee96 p70824    3107 Dec 11 08:28 slurm-1620836.out
-rw-r--r-- 1 trainee96 p70824    3022 Dec 11 08:30 slurm-1620837.out
-rwxr-xr-x 1 trainee96 p70824   31560 Dec 11 08:30 simple-sycl-app


## check if the program compiled

In [50]:
!cat slurm-1620837.out

 
:: initializing oneAPI environment ...
   slurm_script: BASH_VERSION = 4.4.20(1)-release
   args: Using "$@" for setvars.sh arguments: 
:: advisor -- latest
:: ccl -- latest
:: compiler -- latest
:: dal -- latest
:: debugger -- latest
:: dev-utilities -- latest
:: dnnl -- latest
:: dpcpp-ct -- latest
:: dpl -- latest
:: inspector -- latest
:: ipp -- latest
:: ippcp -- latest
:: itac -- latest
:: mkl -- latest
:: mpi -- latest
:: tbb -- latest
:: vtune -- latest
:: oneAPI environment initialized ::
 
**********************************************
output of nvidia-smi
**********************************************
Mon Dec 11 08:30:09 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile 

In [51]:
! ls -ltr 

total 6851
-rw-r--r-- 1 trainee96 p70824  420336 Dec  5 11:46 1_1_welcome_FINAL.pdf
-rw-r--r-- 1 trainee96 p70824 1833724 Dec  7 13:27 1_2_Intel_Devcloud_FINAL.pdf
-rw-r--r-- 1 trainee96 p70824 3678052 Dec  8 11:46 3_adv_sycl_sonersteiner_vienna_FINAL.pdf
-rw-r--r-- 1 trainee96 p70824  268829 Dec  8 13:33 SYCL_Introduction.ipynb
-rw-r--r-- 1 trainee96 p70824  738212 Dec  8 13:33 SYCL_Introduction.pdf
-rw-r--r-- 1 trainee96 p70824     762 Dec 11 06:00 job_vsc5_gpu_run.sh
-rw-r--r-- 1 trainee96 p70824     401 Dec 11 06:14 simple-sycl-app.cpp
drwxrwxr-x 7 trainee96 p70824    4096 Dec 11 06:22 1445-1600-LAB2-advanced-sycl
-rw-r--r-- 1 trainee96 p70824   22316 Dec 11 07:28 1_3_job_submission_vsc5.ipynb
-rw-r--r-- 1 trainee96 p70824    1201 Dec 11 08:27 job_vsc5_compile.sh
-rw-r--r-- 1 trainee96 p70824    3107 Dec 11 08:28 slurm-1620836.out
-rw-r--r-- 1 trainee96 p70824    3022 Dec 11 08:30 slurm-1620837.out
-rwxr-xr-x 1 trainee96 p70824   31560 Dec 11 08:30 simple-sycl-app


# running on a GPU

In [55]:
%%file job_vsc5_gpu_run.sh
#!/bin/bash

#SBATCH --job-name    name
#SBATCH --cluster     vsc5
#SBATCH --reservation training_gpu
#SBATCH --qos         zen3_0512_a100x2_training
#SBATCH --partition   zen3_0512_a100x2
#SBATCH --gres        gpu:1
#SBATCH --time        00:01:00
###########################

spack unload
spack load cuda@11.8.0

source /opt/sw/vsc4/VSC/x86_64/generic/intel/oneapi/setvars.sh 
export LD_LIBRARY_PATH=/gpfs/opt/sw/cuda-zen/spack-0.19.0/opt/spack/linux-almalinux8-zen/gcc-9.5.0/gcc-12.2.0-nu5le4qn6edhcjiocq7wddclrvdj4xfg/lib64:$LD_LIBRARY_PATH    

###########################

echo "starting program simple-sycl-app on the nvidia gpu"
echo "**********************************************"
./simple-sycl-app
echo "**********************************************"


Overwriting job_vsc5_gpu_run.sh


In [56]:
!sbatch job_vsc5_gpu_run.sh

sbatch: Allocating 50.0 % of cpu resources: 64 / 128.
sbatch: Number of tasks adjusted to 64.
Submitted batch job 1620839 on cluster vsc5


In [58]:
!squeue -u $USER

             JOBID            PARTITION     NAME     USER ST       TIME  NODES     NODELIST(REASON)
           1620734    zen3_0512_jupyter vsc5_jh_ trainee9  R    2:37:40      1            n3503-022


In [59]:
!ls -ltr

total 6852
-rw-r--r-- 1 trainee96 p70824  420336 Dec  5 11:46 1_1_welcome_FINAL.pdf
-rw-r--r-- 1 trainee96 p70824 1833724 Dec  7 13:27 1_2_Intel_Devcloud_FINAL.pdf
-rw-r--r-- 1 trainee96 p70824 3678052 Dec  8 11:46 3_adv_sycl_sonersteiner_vienna_FINAL.pdf
-rw-r--r-- 1 trainee96 p70824  268829 Dec  8 13:33 SYCL_Introduction.ipynb
-rw-r--r-- 1 trainee96 p70824  738212 Dec  8 13:33 SYCL_Introduction.pdf
-rw-r--r-- 1 trainee96 p70824     401 Dec 11 06:14 simple-sycl-app.cpp
drwxrwxr-x 7 trainee96 p70824    4096 Dec 11 06:22 1445-1600-LAB2-advanced-sycl
-rw-r--r-- 1 trainee96 p70824   22316 Dec 11 07:28 1_3_job_submission_vsc5.ipynb
-rw-r--r-- 1 trainee96 p70824    1201 Dec 11 08:27 job_vsc5_compile.sh
-rw-r--r-- 1 trainee96 p70824    3107 Dec 11 08:28 slurm-1620836.out
-rw-r--r-- 1 trainee96 p70824    3022 Dec 11 08:30 slurm-1620837.out
-rwxr-xr-x 1 trainee96 p70824   31560 Dec 11 08:30 simple-sycl-app
-rw-r--r-- 1 trainee96 p70824     763 Dec 11 08:31 job_vsc5_gpu_run.sh
-rw-r--r-- 1 trai

In [60]:
!cat slurm-1620839.out

 
:: initializing oneAPI environment ...
   slurm_script: BASH_VERSION = 4.4.20(1)-release
   args: Using "$@" for setvars.sh arguments: 
:: advisor -- latest
:: ccl -- latest
:: compiler -- latest
:: dal -- latest
:: debugger -- latest
:: dev-utilities -- latest
:: dnnl -- latest
:: dpcpp-ct -- latest
:: dpl -- latest
:: inspector -- latest
:: ipp -- latest
:: ippcp -- latest
:: itac -- latest
:: mkl -- latest
:: mpi -- latest
:: tbb -- latest
:: vtune -- latest
:: oneAPI environment initialized ::
 
starting program simple-sycl-app on the nvidia gpu
**********************************************
running program:  simple-sycl-app 
******************************************* 
Selected device: NVIDIA A100-PCIE-40GB
******************************************* 
**********************************************


# running on a CPU

In [61]:
%%file job_vsc5_cpu_run.sh
#!/bin/bash

#SBATCH --job-name    name
#SBATCH --cluster     vsc5
#SBATCH --reservation training_cpu
#SBATCH --nodes       1
#SBATCH --qos         zen3_0512
#SBATCH --partition   zen3_0512
#SBATCH --time        00:01:00
###########################

source /opt/sw/vsc4/VSC/x86_64/generic/intel/oneapi/setvars.sh 

###########################

echo "starting program simple-sycl-app on the cpu"
echo "**********************************************"
./simple-sycl-app
echo "**********************************************"


Writing job_vsc5_cpu_run.sh


In [64]:
!sbatch job_vsc5_cpu_run.sh

Submitted batch job 1620840 on cluster vsc5


In [66]:
!squeue -u $USER

             JOBID            PARTITION     NAME     USER ST       TIME  NODES     NODELIST(REASON)
           1620734    zen3_0512_jupyter vsc5_jh_ trainee9  R    2:39:19      1            n3503-022


In [67]:
!ls -ltr 

total 6853
-rw-r--r-- 1 trainee96 p70824  420336 Dec  5 11:46 1_1_welcome_FINAL.pdf
-rw-r--r-- 1 trainee96 p70824 1833724 Dec  7 13:27 1_2_Intel_Devcloud_FINAL.pdf
-rw-r--r-- 1 trainee96 p70824 3678052 Dec  8 11:46 3_adv_sycl_sonersteiner_vienna_FINAL.pdf
-rw-r--r-- 1 trainee96 p70824  268829 Dec  8 13:33 SYCL_Introduction.ipynb
-rw-r--r-- 1 trainee96 p70824  738212 Dec  8 13:33 SYCL_Introduction.pdf
-rw-r--r-- 1 trainee96 p70824     401 Dec 11 06:14 simple-sycl-app.cpp
drwxrwxr-x 7 trainee96 p70824    4096 Dec 11 06:22 1445-1600-LAB2-advanced-sycl
-rw-r--r-- 1 trainee96 p70824    1201 Dec 11 08:27 job_vsc5_compile.sh
-rw-r--r-- 1 trainee96 p70824    3107 Dec 11 08:28 slurm-1620836.out
-rw-r--r-- 1 trainee96 p70824    3022 Dec 11 08:30 slurm-1620837.out
-rwxr-xr-x 1 trainee96 p70824   31560 Dec 11 08:30 simple-sycl-app
-rw-r--r-- 1 trainee96 p70824     763 Dec 11 08:31 job_vsc5_gpu_run.sh
-rw-r--r-- 1 trainee96 p70824     816 Dec 11 08:31 slurm-1620839.out
-rw-r--r-- 1 trainee96 p70824

In [68]:
!cat slurm-1620840.out

 
:: initializing oneAPI environment ...
   slurm_script: BASH_VERSION = 4.4.20(1)-release
   args: Using "$@" for setvars.sh arguments: 
:: advisor -- latest
:: ccl -- latest
:: compiler -- latest
:: dal -- latest
:: debugger -- latest
:: dev-utilities -- latest
:: dnnl -- latest
:: dpcpp-ct -- latest
:: dpl -- latest
:: inspector -- latest
:: ipp -- latest
:: ippcp -- latest
:: itac -- latest
:: mkl -- latest
:: mpi -- latest
:: tbb -- latest
:: vtune -- latest
:: oneAPI environment initialized ::
 
starting program simple-sycl-app on the cpu
**********************************************
running program:  simple-sycl-app 

UR CUDA ERROR:
	Value:           100
	Name:            CUDA_ERROR_NO_DEVICE
	Description:     no CUDA-capable device is detected
	Function:        operator()
	Source Location: /tmp/tmp.7vgJ2wJCWQ/intel-llvm-mirror/sycl/plugins/unified_runtime/ur/adapters/cuda/platform.cpp:76

******************************************* 
Selected device: AMD EPYC 7713 64-Core Proce

### you can ignore the cuda error! The selected device is:
### AMD EPYC 7713 64-Core Processor